In [ ]:
# !pip install -Uqqq pip --progress-bar off
# !pip install -qqq unstructured==0.8.0 --progress-bar off
# !pip install -qqq langchain --upgrade --progress-bar off
# !pip install -qqq xformers==0.0.20 --progress-bar off
# !pip install  pinecone-client==2.2.2 accelerate==0.21.0  pdfminer.six==20221105  unstructured==0.8.1 bs4==0.0.1
# !pip install chromadb
# !pip install openai
# !pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
import tiktoken
import re
from pathlib import Path
from langchain.chains.question_answering import load_qa_chain
import torch
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseOutputParser
from transformers import (
    AutoTokenizer,
    GenerationConfig,
   TextStreamer,
    pipeline,
)


In [ ]:
"""
#Libraries need to be installed
"""

import re
import base64
import glob
import os
import textwrap
import time
from urllib.parse import urljoin

import chromadb
import pinecone
import requests
import torch
from bs4 import BeautifulSoup

#from constants import CHROMA_SETTINGS
from chromadb.config import Settings
from langchain import HuggingFaceHub, LLMChain
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.document_loaders import (
    DirectoryLoader,
    PDFMinerLoader,
    PyPDFLoader,
    UnstructuredURLLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
from langchain.memory import VectorStoreRetrieverMemory
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate
)
from langchain.prompts import load_prompt
#from project import settings

In [ ]:
# OpenAI
os.environ["OPENAI_API_KEY"]="Enter your OpenAPI Key"
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    temperature = 0.1,
    model_name = "gpt-3.5-turbo",
)

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
import openai

In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
import pinecone
# Connect to your Pinecone
pinecone.init(
    api_key='***********',
    environment='********'
)
index_name = '******'
index = pinecone.Index(index_name)

In [ ]:
#Data Extraction
def extract_urls_from_url(url):
    visited_urls = set()
    try:
        visited_urls.add(url)
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            links = soup.find_all("a", href=True)
            urls = []
            if links is not None:  # Check if l('hi', 'Hello! Home » Blog » What is a “Forever Home”?\nWhat is a “Forever Home”?\nPosted on February 27, 2018 by Emily Cleary\nWhen you think of a “forever home”, what comes to mind? For some, it’s a cozy cottage by the sea, or a spacious farmhouse in the countryside. For others, it’'), ('how are you', "I'm well, thank you. How are you? # Lijst van voetbalinterlands Chili - Mexico\n\nDeze lijst van voetbalinterlands is een overzicht van alle officiële voetbalwedstrijden tussen de nationale team… heden 68 keer tegen elkaar. De eerste ontmoeting, een vriendschappelijke wedstrijd, werd"), ('what are you doing', 'I am a large language model and I am here to answer your questions and help you in any way I can.\n\nDo you have any questions or anything you would like me to do? \ufeffusing System;\nusing System.Collections.Generic;\nusing System.Linq;\nusing System.Text;\nusing System.Threading.Tasks;\nusing System.Windows;\nusing System.Windows.Controls;\nusing System.Windows.Data;\nusing')]
                for link in links:
                    absolute_url = urljoin(url, link["href"])
                    if absolute_url not in visited_urls:
                        urls.append(absolute_url)
            return urls
        else:
            return []
    except requests.exceptions.InvalidSchema:
        print("InvalidSchema: No connection adapters were found.")


In [ ]:
#Pre processing
#NEW ADDITION (start)
import re

def clean_sentence(sentence):
    # Find the index where the sentence starts with numbers or alphabets
    match = re.search(r"[a-zA-Z0-9]", sentence)

    if match:
        # Get the starting index of the matched portion
        start_index = match.start()

        # Remove symbols before the start of the sentence
        cleaned_sentence = sentence[start_index:]
    else:
        # If no numbers or alphabets found, return an empty string
        cleaned_sentence = ""

    # Remove line breaks
    cleaned_sentence = cleaned_sentence.replace("\n", " ")

    # Remove leading and trailing whitespaces
    cleaned_sentence = cleaned_sentence.strip()

    # Convert all alphabets to lowercase
    #cleaned_sentence = cleaned_sentence.lower()

    # Remove consecutive question marks, periods, and apostrophes
    cleaned_sentence = re.sub(r"[@]+", "@", cleaned_sentence)
    cleaned_sentence = re.sub(r"[.]+", ".", cleaned_sentence)
    cleaned_sentence = re.sub(r"[']+", "'", cleaned_sentence)


    # Remove all symbols except period (.), @ mark (@), spaces, and normal numbers
    cleaned_sentence = re.sub(r"[^a-z.A-Z@0-9\s'?:,àéêûî]", "", cleaned_sentence)

    # Replace two or more consecutive spaces with one
    cleaned_sentence = re.sub(r"\s{2,}", " ", cleaned_sentence)

    return cleaned_sentence
#NEW ADDITION[end]



In [ ]:
# print(prompt)
def Pinecone_Upsert(file=None, url=None,csv=None, text=None):
    documents = []
    if file:
        # file = str(settings.BASE_DIR) + file
        file = os.path.join("/content", file)
        print(file)
        loader = PDFMinerLoader(file)
        documents.extend(loader.load())


    if url:
        urls = extract_urls_from_url(url)
        urls = list(set(urls))
        print(urls)
        loader_url = UnstructuredURLLoader(urls=urls)
        documents.extend(loader_url.load())
    if csv:
      csv = os.path.join("/content", csv)
      print(csv)
      loader = CSVLoader(csv, encoding='ISO-8859-1')
      documents.extend(loader.load())
    if text:
      loader = TextLoader(text)
      documents.extend(loader.load())



    if len(documents) > 0:
        print("The loaded document: ",documents)
#NEW ADDITION [start]
        for _index in range(len(documents)):
            # Step 1: Access the page_content attribute of the custom object
            document = documents[_index]
            page_content = document.page_content
            #print(page_content)

            # Step 2: Perform transformations on the extracted text
            transformed_content = clean_sentence(page_content)

            #print(transformed_content)

            # Step 3: Update the page_content attribute with the transformed text
            document.page_content = transformed_content
            #print(document)

            # Step 4: Reconstruct the original structure with square brackets
            documents[_index] = document
            print("Transformed Document Final: ", documents[_index])

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500,
                                                       chunk_overlap=200
                                                       )
        # print("Splitter Address in meory: ", text_splitter)
#NEW ADDITION[end]


        texts = text_splitter.split_documents(documents)
        print("Splitted document: ",texts)
        docsearch = Pinecone.from_existing_index(index_name, embeddings)
        docsearch.add_documents(texts)



Upserting single document

In [ ]:
def Upsert_text(text):
  db = Pinecone.from_existing_index(index_name, embeddings)
  db.add_texts(text)

In [ ]:
path = "Enter File Path"
        #For PDF run this
Pinecone_Upsert(file=pdf_file_path)
        #For URL run this
#Pinecone_Upsert(url=pdf_file_path)
        #For CSV run this
#Pinecone_Upsert(csv=pdf_file_path)
        #For paragraph run this
#Pinecone_Upsert(text=pdf_file_path)

Delete Index

In [ ]:
# Warning! Only run this cell if you want to delete all the data from the pinecone database. Once run all the data will be deleted and will never reversed.

# index.delete(delete_all = True)

Upserting Multiple Document

In [ ]:
import os
# Define the folder path containing your PDF files
pdf_folder_path = "Enter File Path"

# List all files in the folder
pdf_files = os.listdir(pdf_folder_path)

# Iterate through the PDF files and upsert them
for pdf_file in pdf_files:
    # Construct the full path to the PDF file
    pdf_file_path = os.path.join(pdf_folder_path, pdf_file)

    # Replace file with url, csv, text to upsert that specific thing
    Pinecone_Upsert(file=pdf_file_path)

    # Optionally, you can print a message to indicate which PDF is being processed
    print(f"Upserted PDF: {pdf_file}")


In [ ]:
text_field = "text"
db = Pinecone(index, embeddings.embed_query, text_field)

In [ ]:
retriever = db.as_retriever(search_type="mmr"
)
query = "What is epnyonjuraprangins?"
#db.similarity_search(query, k=3)
retriever.get_relevant_documents(query)

**ChatBot**

In [ ]:
DEFAULT_TEMPLATE = """
### Instruction: Your name is 'Jarvis' and the name of the customer is {username}. Please introduce yourself as 'Jarvis, Information Assistant'. Use the user's name occasionally in your responses, but ensure to introduce yourself only in the first response.
Utilize the chat history and available information in "{context}" to answer the question.
Remember, the user is looking for assistance, so keep your responses natural, concise, accurate, and informative. If you are uncertain about a query or if the user asked something which is unidentified by you, prompt the user to rephrase it. 
When greeted, respond briefly and amiably.
{chat_history}
### Input: {question}
### Response:
"""

prompt = PromptTemplate (
        input_variables = ["context", "question", "chat_history", "username"],
        template = DEFAULT_TEMPLATE,
    )
memory = ConversationBufferMemory(
        memory_key = "chat_history",
        human_prefix = "### Input",
        ai_prefix = "### Response",
        input_key = "question",
        output_key = "output_text",
        return_messages = False,
    )
username = "Faiq"
chain = load_qa_chain(
        llm,
        chain_type="stuff",
        prompt=prompt,
        memory=memory,
        verbose=False,
    )
db = Pinecone(index, embeddings, "text")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
while True:
  retriever = db.as_retriever(search_type="mmr")
  user_input=input("Enter: ")
  if user_input.lower() in ["bye", "goodbye", "quit", "exit"]:
    print(" Bye! It was nice chatting with you.")
    break
  #docs = db.similarity_search(user_input)
  docs = retriever.get_relevant_documents(user_input)
  response = chain.run({"input_documents":docs, "question": user_input, "username":username})
  print(response)